In [1]:
import urllib.request
import json
import numpy as np
import pandas as pd
import altair
import pprint
import os, subprocess, sys,codecs, locale
import re
import traceback
from collections import namedtuple
import datetime
import pdfplumber
import pickle
import pandas as pd
import openai
from pprint import pprint
from aocutils.datascience import save, load
token = open('.tokens/test.txt','r').read()
openai.organization = ""
openai.api_key = token

# openai.Model.list()

In [2]:
from pathlib import Path
df = pd.read_pickle("moties_for_llm.pickle")
print(len(df))
if Path("motie_summaries.pickle").exists():
    print('Summary file found')
    summaries = pd.read_pickle("motie_summaries.pickle").drop_duplicates('doc_Id',keep='last')
    df = pd.merge(df, summaries, on='doc_Id', how='outer')
    print(f"Amount of moties {len(df)} without summary {df['Summary'].isna().sum()}")
df['len'] = df['Prompt'].str.len()
# df = df.sample(frac=1)

9008
Summary file found
Amount of moties 9008 without summary 1


In [3]:
char2token = 0.4
summarize_factor = 0.21
# calculate amount of tokens
test = df.groupby(['Indienende_partij'])['len'].aggregate([sum, len])
test['tokens'] = round(test['sum'] * char2token * summarize_factor,0)
test['tokenspermotie'] = round(test['tokens'] / test['len'])
test['numsummarysummaries'] = test['len'] // 18
test

,sum,len,tokens,tokenspermotie,numsummarysummaries
Indienende_partij,,,,,
50PLUS,2248,4,189.0,47.0,0
BBB,306920,427,25781.0,60.0,23
BIJ1,120682,125,10137.0,81.0,6
CDA,560330,638,47068.0,74.0,35
ChristenUnie,392015,475,32929.0,69.0,26
D66,619013,756,51997.0,69.0,42
DENK,303575,507,25500.0,50.0,28
FVD,118170,220,9926.0,45.0,12
Fractie Den Haan,102134,138,8579.0,62.0,7


In [4]:
token_count = 0
summary_count = 0
already_done = 0
errors = 0
model = "gpt-3.5-turbo"
def get_sum(row):
    global token_count, summary_count, already_done, errors
    if 'Summary' not in row.index or pd.isna(row.Summary):
        try:
            completion = openai.ChatCompletion.create(
            model = model,
            messages=[
                {"role": "system", "content": "Je bent heel goed in het samenvatten van moties. Je vat moties samen in maximaal 2 zinnen. Dat de Kamer de regering verzoekt laat je weg."},
                {"role": "user", "content": row.ClippedText},
            ]
            )
            token_count += completion['usage']['total_tokens']
            summary_count += 1
            if not summary_count % 10:
                print(summary_count)
            return {'doc_Id': row.doc_Id,
                    'ClippedTextforSummary':row.ClippedText,
                    'SummaryModel': model,
                    'SummaryDateRetrieved' : datetime.date.today(),
                    'Summary': completion.choices[0].message.content, 
                    'SummaryTokens': completion['usage']['completion_tokens']}
        except Exception as e:
            print('call didnt go well',e)
            errors += 1
        
    
    already_done += 1
    return {'doc_Id': row.doc_Id,
        'ClippedTextforSummary':row.ClippedTextforSummary,
        'SummaryModel': row.SummaryModel,
        'SummaryDateRetrieved' : row.SummaryDateRetrieved,
        'Summary': row.Summary, 
        'SummaryTokens': row.SummaryTokens}

output = df.apply(get_sum, axis='columns', result_type='expand')
summaries = pd.merge(summaries, output, how='outer').drop_duplicates('doc_Id', keep='last')
save(summaries, 'motie_summaries.pickle')
print(f'{summary_count} summaries downloaded, with {token_count} total tokens. {already_done} were already summarized earlier. {errors} errors occurred. Total summaries now {len(summaries)}')
df = pickle.load(open("moties_for_llm.pickle","rb"))
df = pd.merge(df, summaries, on='doc_Id', how='outer')
save(df, 'moties_for_llm_incl_moties.pickle')


1 summaries downloaded, with 358 total tokens. 9007 were already summarized earlier. 0 errors occurred. Total summaries now 9008


### Using GPT-4 to summarize all motie summaries per party

In [1]:
import urllib.request
import json
import numpy as np
import pandas as pd
import altair
import pprint
import os, subprocess, sys,codecs, locale
import re
import traceback
from collections import namedtuple
import datetime
import pdfplumber
import pickle
import pandas as pd
import openai
from pprint import pprint
from aocutils.datascience import save, load

openai.organization = ""
openai.api_base = "https://openrouter.ai/api/v1"
openroutertoken = open('.tokens/test2.txt','r').read()
openai.api_key = openroutertoken
OPENROUTER_REFERRER = "https://github.com/jvanelteren/kamermoties"

# openai.Model.list()
df = load('moties_for_llm_incl_moties.pickle')

In [2]:
# With the summaries, the amount of tokens is reduced with ~80%
# this is roughly the cost with gpt 4
df['SummaryTokens'].sum() / 1000 * 0.06

20.08152

In [3]:
df = df.sort_values(['Indienende_partij'])
df['CumulativeTokens'] = df.groupby('Indienende_partij')['SummaryTokens'].cumsum()

In [4]:
import math
max_tokens_per_summary = 3300
tokens_party = df.groupby('Indienende_partij')['SummaryTokens'].sum().reset_index()

tokens_party['Amount_tokens_per_summary'] = tokens_party['SummaryTokens'] // ((tokens_party['SummaryTokens'] // max_tokens_per_summary) + 1) + 10
tokens_party


,Indienende_partij,SummaryTokens,Amount_tokens_per_summary
0,BBB,17274.0,2889.0
1,BIJ1,3639.0,1829.0
2,CDA,27174.0,3029.0
3,ChristenUnie,20503.0,2939.0
4,D66,32292.0,3239.0
5,DENK,15465.0,3103.0
6,FVD,5381.0,2700.0
7,Fractie Den Haan,4865.0,2442.0
8,GroenLinks,25890.0,3246.0
9,Groep Van Haga,14890.0,2988.0


In [5]:
# remove not relevant parties
peilingen = 'VVD D66 PVV CDA SP PvdA GroenLinks FVD PvdD ChristenUnie Volt JA21 SGP DENK BBB Omtzigt'.split()
peilingen += ['Omtzicht']
for p in df['Indienende_partij'].unique():
    # deze partijen neem ik niet mee in de analyses
    if p not in peilingen:
        print(p)
print(len(df))
df = df[df['Indienende_partij'].isin(peilingen)]        
print(len(df))

BIJ1
Fractie Den Haan
Groep Van Haga
Gündogan
6017
5481


In [6]:

token_count = 0
summary_count = 0
errors = 0
def get_final_summary(text):
    global token_count, errors, summary_count
    try:
        completion = openai.ChatCompletion.create(
        model="openai/gpt-4-32k", #gpt-4-32k
        headers={"HTTP-Referer": OPENROUTER_REFERRER},
        messages=[
              {"role": "system", "content": "Je analyseert principes van een politieke partij. De input zijn samenvattingen van moties ingediend door de partij, gescheiden door '|'. Je geeft in 1 paragraaf aan wat de principes van de partij zijn. De laatste zin bevat enkel drie kernprincipes, gescheiden door een komma"},
              {"role": "user", "content": text},
          ]
        )
        token_count += completion['usage']['total_tokens']
        summary_count += 1
        if not summary_count % 10:
            print(summary_count)
        return completion.choices[0].message.content
    except Exception as e:
        print('call didnt go well',e)
        errors += 1
        return ''


In [14]:
finals = []
# some parties need reduction of number of moties
party_to_reduce = ['VVD', 'D66']
for partij, group in df.groupby('Indienende_partij'):
        moties_amount = len(group)
        if partij in party_to_reduce:
                moties_amount = moties_amount * 9 // 10
        motie_samenvattingen = ' | '.join(group['Summary'][:reduced_amount])
        result = get_final_summary(' | '.join(group['Summary']))
        print(result)

        finals.append({
                'Partij' : partij,
                'Summary': result,
                'Moties': moties_amount,
                'Motie_samenvattingen': motie_samenvattingen
        })
df_final = pd.DataFrame(finals)
save(df_final, 'final_summaries.pickle')
print(f'Total tokens {token_count}, summaries {summary_count}, errors {errors}')

partij BBB Gegeven de wenselijkheid van het stimuleren van het gebruik van zonnepanelen op daken en de tijdelijkheid van een procentuele terugleveringswaarde van 80%, verzoekt de Kamer de regering om deze waarde voor minimaal tien jaar vast te leggen en wijzigingen minimaal drie jaar voor de ingangsdatum te communiceren. | De Kamer overweegt dat het productieplafond niet gerelateerd is aan de waterkwaliteit en verzoekt daarom de regering om het productieplafond te beperken tot de maximering die is beschreven in de Meststoffenwet met de productiecijfers over het jaar 2002. | Monitor oversterfte door onderkoeling en informeer de Kamer voor het meireces over de totale cijfers. | Er wordt verzocht dat het kabinet met de NPO in gesprek gaat over het bevorderen van regionale programmering en diversiteit in de regio's voor talkshows. | Er moet een Nationaal Koudeplan worden opgesteld door het RIVM en ingezet worden bij langere periodes van koud weer, vanwege de kwetsbaarheid van ouderen en an

### Interpretation

In [9]:
import urllib.request
import json
import numpy as np
import pandas as pd
import altair
import pprint
import os, subprocess, sys,codecs, locale
import re
import traceback
from collections import namedtuple
import datetime
import pdfplumber
import pickle
import pandas as pd
import openai
from pprint import pprint
from aocutils.datascience import save, load

openai.organization = ""
openai.api_base = "https://openrouter.ai/api/v1"
openroutertoken = open('.tokens/test2.txt','r').read()
openai.api_key = openroutertoken
OPENROUTER_REFERRER = "https://github.com/jvanelteren/kamermoties"

# openai.Model.list()
df = load('moties_for_llm_incl_moties.pickle')

df_final = load('final_summaries.pickle')



In [11]:
df_final = df_final.sample(frac=1)

In [21]:
for idx, row in enumerate(df_final.iterrows()):
    if row[1].values[0] == 'CDA':
        print('Partij', str(idx+1), row[1].values[0])
        print(row[1].values[3])

Partij 10 CDA
Het kabinet wordt verzocht de maatschappelijke kosten-batenanalyse niet langer als enige besluitvormingsinstrument in het infrastructuurbeleid te gebruiken en te onderzoeken hoe deze analyse nog waarde kan hebben. Tevens wordt het kabinet verzocht het eenzijdig rendementsdenken te schrappen uit toekomstige analyses en inzichtelijk te maken hoe de analyses worden gebaseerd op bereikbaarheidsnormen. | Maak de groene waterstofroutekaart voor de raffinagesector volwaardig en bied langetermijnzekerheid aan marktpartijen door na 2025 openstelling te bieden voor groene waterstof in Nederland. | Neem bij een bod van de regio voor de Bodegravenboog een werkende parallelstructuur inclusief extra op- en afrit bij Bodegraven mee en stel een routekaart op voor een veilige ongelijkvloerse fietsoversteek bij de op- en afritten van de N11 bij Bodegraven binnen drie jaar. | De Kamer verzoekt de regering periodiek inzicht te geven in de samenstelling van referentiebudgetten van sociale min

In [13]:
df_final

,Partij,Summary,Moties,Motie_samenvattingen
8,Omtzigt,De politieke partij richt zich op diverse gebi...,131,Vóór 1 december wil de Kamer een AMvB van de r...
12,SGP,De politieke partij zet zich in voor een breed...,163,Het patenteren van op klassieke wijze veredeld...
5,FVD,Deze politieke partij lijkt een sterke nadruk ...,122,Opheffing van de sancties tegen Syrië om human...
4,DENK,De politieke partij in kwestie heeft een sterk...,284,Intrekking van het wetsvoorstel Wijziging van ...
14,VVD,Uit de moties blijkt dat de politieke partij z...,466,- Oproepen om de systeemkosten van de vrachtwa...
13,SP,De partij wiens moties zijn geanalyseerd zet z...,589,Geen winstuitkeringen in de kinderopvangsector...
3,D66,Deze politieke partij lijkt een sterke focus t...,473,Onderzoek knelpunten publieke onderwijsinstell...
10,PvdA,Uit de ingediende moties blijkt een diversitei...,447,Kabinet in gesprek gaan met betrokkenen over e...
0,BBB,De partij zet zich in voor beleid dat duurzaam...,309,Gegeven de wenselijkheid van het stimuleren va...
1,CDA,De politieke partij waarvan de moties zijn gea...,413,Het kabinet wordt verzocht de maatschappelijke...


In [7]:
par = [row[1].values[0] for row in df_final.iterrows()]
import numpy
numpy.random.shuffle(par)
par

['JA21',
 'ChristenUnie',
 'CDA',
 'PVV',
 'PvdD',
 'SP',
 'DENK',
 'D66',
 'PvdA',
 'FVD',
 'GroenLinks',
 'VVD',
 'SGP',
 'Omtzigt',
 'BBB',
 'Volt']